In [ ]:
# ------ Copyright (C) 2020 University of Strathclyde and Author ------
# ---------------- Author: Francesco Marchetti ------------------------
# ----------- e-mail: francesco.marchetti@strath.ac.uk ----------------


In [2]:
!pip install -q SharedArray
!pip install -q pypianoroll==0.5.3
#!pip install wandb

import sys
import os
import numpy as np

%tensorflow_version 1.x
import tensorflow as tf

import SharedArray as sa
from musegan.bmusegan.models import GAN, RefineGAN, End2EndGAN
from config_transfer import EXP_CONFIG, DATA_CONFIG, MODEL_CONFIG, TRAIN_CONFIG
from config_transfer import TF_CONFIG
#import wandb
#wandb.init()


     |████████████████████████████████| 5.6MB 11.1MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
TensorFlow 1.x selected.
Found GPU at: /device:GPU:0
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2138112/45929032 bytes (4.7%)6275072/45929032 bytes (13.7%)10502144/45929032 bytes (22.9%)14573568/45929032 bytes (31.7%)18702336/45929032 bytes (40.7%)22872064/45929032 bytes (49.8%)27041792/45929032 bytes (58.9%)31096832/45929032 bytes (67.7%)35233792/45929032 bytes (76.7%)39387136/45929032 by

In [3]:
def load_data():
    """Load and return the training data."""
    print('[*] Loading data...')
    DATA_CONFIG['training_data'] = '/bmusegan/training_data/final-balanced-scottish-midi-data.npy'
    if DATA_CONFIG['training_data_location'] == 'sa':
        x_train = sa.attach(DATA_CONFIG['training_data'])
    elif DATA_CONFIG['training_data_location'] == 'hd':
        x_train = np.load(DATA_CONFIG['training_data'])
    x_train = x_train.reshape(
        -1, MODEL_CONFIG['num_bar'], MODEL_CONFIG['num_timestep'],
        MODEL_CONFIG['num_pitch'], MODEL_CONFIG['num_track']
    )
    print('Training set size:', len(x_train))
    return x_train

def pretrain():
    """Create and pretrain a two-stage model"""
    x_train = load_data()
    with tf.Session(config=TF_CONFIG) as sess:
        gan = GAN(sess, MODEL_CONFIG)
        gan.init_all()
        if EXP_CONFIG['pretrained_dir'] is not None:
            gan.load_latest(EXP_CONFIG['pretrained_dir'])
        gan.train(x_train, TRAIN_CONFIG)

def train():
    """Load the pretrained model and run the second-stage training"""
    x_train = load_data()
    with tf.Session(config=TF_CONFIG) as sess:
        gan = GAN(sess, MODEL_CONFIG)
        gan.init_all()
        gan.load_latest(EXP_CONFIG['first_stage_dir'])

        refine_gan = RefineGAN(sess, MODEL_CONFIG, gan)
        refine_gan.init_all()
        if EXP_CONFIG['pretrained_dir'] is not None:
            refine_gan.load_latest(EXP_CONFIG['pretrained_dir'])
        refine_gan.train(x_train, TRAIN_CONFIG)

def train_end2end():
    """Create and train an end-to-end model"""
    x_train = load_data()
    with tf.Session(config=TF_CONFIG) as sess:
        end2end_gan = End2EndGAN(sess, MODEL_CONFIG)
        end2end_gan.init_all()
        if EXP_CONFIG['pretrained_dir'] is not None:
            end2end_gan.load_latest(EXP_CONFIG['pretrained_dir'])
        end2end_gan.train(x_train, TRAIN_CONFIG)

if __name__ == '__main__':
    print("Start experiment: {}".format(EXP_CONFIG['exp_name']))
    if not EXP_CONFIG['two_stage_training']:
        train_end2end()
    elif TRAIN_CONFIG['training_phase'] == 'pretrain':
        pretrain()
    elif TRAIN_CONFIG['training_phase'] == 'train':
        train()

Start experiment: haggis_balancedDataset_100Epochs_transferLearning_9
[*] Loading data...
Training set size: 1047
[*] Building GAN...



Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


Instructions for updating:
Use `tf.keras.layers.Conv3DTranspose` instead.

Instructions for updating:
Use `tf.keras.layers.Conv3D` instead.


=============================== Model Statistics ===============================

Number of parameters: 3735737
Number of parameters in G: 1580440
Number of parameters in D: 2155297
================================ Model Summary =================================
----------------------------------- < GAN > ------------------------------------
--------------------